# **Sustainable logistics project**
We decided to work on tourism in Switzerland. Switzerland being a relatively expensive country for a foreigner due to the high standard of living here, it seems important to us to be able to propose solutions to visit this magnificent country at a lower cost. The problems of travel time and distance traveled are also to be considered insofar as traveling while taking care of one's footprint on the environment is important in view of the current climate situation but also to preserve the Swiss environment which is still very well preserved compared to other countries. In order to optimize this, we have designed different algorithms seen during Sustainable Logistics to allow travelers to visit Switzerland at a lower cost but also to reduce their impact on the climate.
We started by establishing a top 10 of destinations to discover in Switzerland thanks to a website on which the best tourist destinations in the country are listed (Cap-Voyage).

**According to CapVoyages, here are the 10 best places to visit in Switzerland.**

1.   Observation platform of Gornergrat
2.   Chillon's castle
3.   Jungfraujoch
4.   Schweizerischer national park
5.   Lemanic Lake in Lausanne
6.   Lugano's town
7.   Berne's town
8.   Lucerne's town
9.   Rhin's waterfalls
10.  Zurich's town


## **1. Different TSP models to optimize**
The first part of our project will be about trying to reduce the time/distance to visit all those 10 places.

First, we will need to build our matrix with the distances of all the places between each others.

- Distance & Environmental impact (car + train) **fait par Arthur**
- Time (car + train)
- Cost (car + train)

### **1.1 Optimize distance to visit the top 10 destinations**

#### **1.1.1 By car**

In [ ]:
#Package installation and importation
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#Model and constraints variables creation 
def create_data_model():
    data = {}
    data['distance_matrix'] = [
    [0, 170, 53, 184, 236, 251, 136, 133, 233, 210], # Observation platform of Gornergrat
    [170, 0, 160, 116, 76, 227, 104, 177, 306, 207], # Chillon's castle
    [53, 160, 0, 200, 222, 228, 56, 73, 303, 131], # Jungfraujoch
    [184, 116, 200, 0, 215, 361, 174, 221, 137, 237], # Schweizerischer National Park
    [236, 76, 222, 215, 0, 187, 76, 154, 419, 220], # Lemanic Lake in Lausanne
    [251, 227, 228, 361, 187, 0, 161, 166, 220, 195], # Lugano's town
    [136, 104, 56, 174, 76, 161, 0, 44, 216, 101], # Berne's town
    [133, 177, 73, 221, 154, 166, 44, 0, 242, 51], # Lucerne's town
    [233, 306, 303, 137, 419, 220, 216, 242, 0, 53], # Rhin's waterfalls
    [210, 207, 131, 237, 220, 195, 101, 51, 53, 0] # Zurich's town
    ]
    data['num_vehicles'] = 1
    data['depot'] = 9
    return data

#Printing out the results of a solution to the Vehicle Routing Problem
def print_solution(manager, routing, solution):
    print('Objective: {} km'.format(solution.ObjectiveValue()))
    #Start looping over the nodes visited by the first vehicle
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    #Loop until the first vehicle returns to its depot
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    #Add the last node (depot) to the string
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    #Add the total distance traveled by the first vehicle to the string
    plan_output += 'Route distance: {}km\n'.format(route_distance)
    print(plan_output)
    
#Solving the Vehicle Routing Problem (VRP) using the Google OR-Tools library
def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    #Define a distance callback function for the VRP
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    #Solving the VRP using the routing model and search parameters
    solution = routing.SolveWithParameters(search_parameters)
    #If a solution is founded, print it out using the print_solution function
    if solution:
        print_solution(manager, routing, solution)
#Launches the main function
if __name__ == '__main__':
    main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Objective: 1023 km
Route for vehicle 0:
 9 -> 7 -> 0 -> 2 -> 6 -> 5 -> 4 -> 1 -> 3 -> 8 -> 9
Route distance: 1023km



With this first basic function and analysis, we see that according to the distance criterion, and assuming that travellers start from Zurich airport for ease, the best route to take is the following:
Zurich => Lucerne => Observation platform of Gornergrat => Jungfraujoch => Bern => Lugano => Lausanne => Chillon's Castle => Schweizerischer national park => Rhine's waterfalls => Zurich

This route is 1023km long and is done exclusively by car. 

Given that a car journey emits on average 121g of CO2 per kilometre, this journey represents an impact of 123.783kg of CO2.

We can therefore compare it with the train in order to give the tourist the possibility to choose his mode of transport according to the emissions of the latter.


#### **1.1.2 By train**

In [ ]:
#Package installation and importation
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#Model and constraints variables creation 
def create_data_model():
    data = {}
    data['distance_matrix'] = [
    [0, 139, 88, 340, 218, 392, 173, 234, 331, 244], # Observation platform of Gornergrat
    [139, 0, 186, 201, 29, 305, 103, 164, 261, 174], # Chillon's castle
    [88, 186, 0, 269, 207, 381, 162, 223, 320, 233], # Jungfraujoch
    [340, 201, 269, 0, 172, 134, 207, 195, 182, 193], # Schweizerischer National Park
    [218, 29, 207, 172, 0, 276, 74, 135, 232, 145],  # Lemanic Lake in Lausanne
    [392, 305, 381, 134, 276, 0, 280, 248, 222, 197], # Lugano's town
    [173, 103, 162, 207, 74, 280, 0, 61, 158, 72],    # Berne's town
    [234, 164, 223, 195, 135, 248, 61, 0, 211, 52],   # Lucerne's town
    [331, 261, 320, 182, 232, 222, 158, 211, 0, 159], # Rhin's waterfalls
    [244, 174, 233, 193, 145, 197, 72, 52, 159, 0]    # Zurich's town
]
    data['num_vehicles'] = 1
    data['depot'] = 9
    return data

#Printing out the results of a solution to the Vehicle Routing Problem
def print_solution(manager, routing, solution):
    print('Objective: {} km'.format(solution.ObjectiveValue()))
    #Start looping over the nodes visited by the first vehicle
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    #Loop until the first vehicle returns to its depot
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    #Add the last node (depot) to the string
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    #Add the total distance traveled by the first vehicle to the string
    plan_output += 'Route distance: {}km\n'.format(route_distance)
    print(plan_output)

#Solving the Vehicle Routing Problem (VRP) using the Google OR-Tools library
def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    #Define a distance callback function for the VRP  
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    #Solving the VRP using the routing model and search parameters
    solution = routing.SolveWithParameters(search_parameters)
    #If a solution is founded, print it out using the print_solution function
    if solution:
        print_solution(manager, routing, solution)

#Launches the main function
if __name__ == '__main__':
    main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Objective: 1218 km
Route for vehicle 0:
 9 -> 7 -> 6 -> 2 -> 0 -> 1 -> 4 -> 3 -> 5 -> 8 -> 9
Route distance: 1218km



By taking the train, travellers will therefore have to travel just over 200 kilometres more to visit all 10 tourist sites. However, the travel time may be more advantageous, which we will address in a later part of this project. What is interesting here is that the order of visiting the destinations is different. Indeed, using the train as a means of transport, travellers will take the following route in case they want to minimise their travel distance:

Zurich => Lucerne => Bern => Jungfraujoch => Observation platform of Gornergrat => Chillon's castle => Lausanne => Schweizerischer national park => Lugano => Rhine's waterfalls => Zurich

This route is exactly 1218km long.

Given that a train journey on average 37g of CO2 per kilometre, this journey represents an impact of 45.066kg of CO2 but for a whole train ! Even with only one wagon, this is significantly less than a car.


#### **1.1.3 Conclusion**

Travelling by car is way shorter, more practical and has surely more advantages. But by comparing the CO2 emissions of both transportations mode, it's clear that train is much more ecological.

Depending on consumers preferences and way of consuming, this analysis can provide him different information about his emissions and the distance he has to travel in order to make the whole tour. 


### **1.2 Optimize time to visit the top 10 destinations.**
Both of the following models are applying exactly the same principle as the first two models we have seen here above.
Except the fact that we are now working with time and not with distances anymore


#### **1.2.1** By car 

In [1]:
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Initialisation des données

def create_data_model():
    data = {}
    data['car_times'] = [
        [0, 120, 270, 240, 60, 210, 60, 90, 330, 150],# Observation platform of Gornergrat
        [120, 0, 270, 240, 60, 330, 60, 120, 360, 120], # Chillon's castle
        [270, 270, 0, 180, 210, 450, 90, 120, 330, 150], # Jungfraujoch
        [240, 240, 180, 0, 270, 420, 60, 90, 270, 90], # Schweizerischer National Park
        [60, 60, 210, 270, 0, 360, 30, 60, 210, 30], # Lemanic Lake in Lausanne
        [210, 330, 450, 7, 360, 0, 330, 390, 10, 210], # Lugano's town
        [60, 60, 90, 60, 30, 330, 0, 30, 150, 30], # Berne's town
        [90, 120, 120, 90, 60, 390, 30, 0, 210, 30], # Lucerne's town
        [330, 360, 330, 270, 210, 10, 150, 210, 0, 90], # Rhin's waterfalls
        [150, 120, 150, 90, 30, 210, 30, 30, 90, 0]   # Zurich's town
    ]
    data['num_vehicles'] = 1
    data['depot'] = 9
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} min'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}km\n'.format(route_distance)

def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['car_times']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['car_times'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(manager, routing, solution)
        
if __name__ == '__main__':
    main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Objective: 880 min
Route for vehicle 0:
 9 -> 7 -> 3 -> 2 -> 6 -> 1 -> 4 -> 0 -> 5 -> 8 -> 9



If someone wants to visit the 10 tourist places by car and minimize the travel time between each place, one should make the following tour: Zurich, city of Lucerne, Schweizerischer National Park, Jungfraujoch, city of Bern, castle of Chillon, Lake Geneva in Lausanne, observation platform of Gornergrat, city of Lugano, Rhine Falls and finally return to Zurich airport. This route takes into account the constraint of starting and finishing in Zurich, the city with the largest airport in Switzerland. The total travel time for this tour would be 14 hours and 50 minutes.

#### **1.2.2** By train 
The following analysis is exactly the same as the first we have seen in this part but this time, the given traveltimes are related to train and not to car anymore.

In [2]:
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Initialisation des données

def create_data_model():
    data = {}
    data['train_times'] = [
        [  0, 210, 180, 390, 150, 330, 150, 210, 360, 210], #Observation platform of Gornergrat
        [210,   0, 210, 360,  90, 360,  90, 150, 270,  90], #Chillon's castle
        [180, 210,   0, 390, 210, 390, 150, 210, 330, 150], #Jungfraujoch
        [390, 360, 390,   0, 300, 480, 210, 300, 420, 210], #Schweizerischer National Park
        [150,  90, 210, 300,   0, 270,  60,  90, 180,  60], #Lemanic Lake in Lausanne
        [330, 360, 390, 480, 270,   0, 300, 330, 390, 150], #Lugano's town
        [150,  90, 150, 210,  60, 300,   0,  90, 180,  60], #Berne's town
        [210, 150, 210, 300,  90, 330,  90,   0, 270,  30], #Lucerne's town
        [360, 270, 330, 420, 180, 390, 180, 270,   0, 150], #Rhin's waterfalls
        [210,  90, 150, 210,  60, 150,  60,  30, 150,   0]  #Zurich's town
    ]
    data['num_vehicles'] = 1
    data['depot'] = 9
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} min'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}km\n'.format(route_distance)

def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['train_times']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['train_times'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(manager, routing, solution)
        
if __name__ == '__main__':
    main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Objective: 1860 min
Route for vehicle 0:
 9 -> 7 -> 3 -> 6 -> 8 -> 4 -> 1 -> 2 -> 0 -> 5 -> 9



If we want to optimise the travel time by train, the route minimising the total travel time is not the same and it is longer. The shortest one goes in order through Zurich, city of Lucerne, Schweizerischer National Park, city of Berne, Rhine Falls, Lake Geneva in Lausanne, Chillon Castle, Jungfraujoch, Observation platform of Gornergrat, city of Lugano and finally back to Zurich, always with the constraint of starting and ending at Kloten airport. This time the total travel time is 31 hours.

**Assumptions:**
- The model doesn"t consider traffic jams and potential train delays
- We consider that a tourists are travelling either by train or by car, not with both means.

#### **1.2.3** Conclusion 

The train journey is more than twice as long as the car journey. A traveller who wants to minimise his total travel time would therefore be well advised to choose the car as a mean of transport, as this would save him 16 hours and 20 minutes. 

Travelling by car would thus cost around 580 CHF considering all our assumptions and hypothesis. The following analysis concern train and will allow us to a make a final comparison afterwise.

### **1.3 Optimize cost by train.**

This will allow us to compare the cost of making the travel by car and by train. 

By train, a traveler would have 2 options : 
- Paying full price each fare
- Paying half price by buying a half-fare reduction card for 185 CHF.

We will compare these two alternatives to see if which is cheaper and if it is better than using a car. 

**Assumption**:
- We consider the price of half-fare train tickets

In [7]:
#Package installation and importation
!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    data = {}
    data['train_cost_matrix'] = [
      [0, 3700, 4150, 8450, 4050, 5350, 4500, 5850, 6850, 6250], # Observation platform of Gornergrat (Zermatt)
      [3700, 0, 3700, 6200, 650, 5600, 2100, 3600, 4400, 4000], # Chillon's castle (Montreux)
      [4150, 3700, 0, 5800, 2800, 4200, 1400, 1650, 4000, 2550], # Jungfraujoch (Interlaken Ost)
      [8450, 6200, 5800, 0, 6000, 5600, 5000, 3700, 3600, 3150], # Schweizerischer National Park (Zernez puis Bus à 2CHF)
      [4050, 650, 2800, 6000, 0, 5400, 1700, 3250, 4200, 3700], # Lemanic Lake in Lausanne
      [5350, 5600, 4200, 5600, 5400, 0, 4400, 3050, 3800, 3250], # Lugano's town
      [4500, 2100, 1400, 5000, 1700, 4400, 0, 1950, 3050, 2550], # Berne's town
      [5850, 3600, 1650, 3700, 3250, 3050, 1950, 0, 3700, 2500], # Lucerne's town
      [6850, 4400, 4000, 3600, 4200, 3800, 3050, 3700, 0, 2320], # Rhin's waterfalls (Neuhasen Rheinfall)
      [6250, 4000, 2550, 3150, 3700, 3250, 2550, 2500, 2320, 0] # Zurich's town
  ]

    data['num_vehicles'] = 1
    data['depot'] = 9
    return data

def print_solution(manager, routing, solution):
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_cost = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_cost += routing.GetArcCostForVehicle(previous_index, index, 0)
    route_cost = route_cost/100
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    plan_output += 'Route cost: {}km\n'.format(route_cost)
    print(plan_output.replace('km', 'CHF'))
    

def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['train_cost_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['train_cost_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(manager, routing, solution)        

main()   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Route for vehicle 0:
 9 -> 5 -> 7 -> 2 -> 0 -> 1 -> 4 -> 6 -> 8 -> 3 -> 9
Route cost: 279.5CHF



Travelling by train will thus cost 279.5 CHF with half-fare train tickets. To access such prices, a 185 CHF card needs to be buyed. Therefore, it would cost 459.5 CHF in total. 
It stays cheaper than travelling with full-fare tickets which would have cost 559 CHF.

**Conclusion**

We can see that, by travelling by train is in any case cheaper than travelling by car.
However, we find it surprising that no special options for tourists are offered by the Swiss railway. 
Indeed, the cheapest option is to buy an annual half-fare card to reduce the price of tickets. 
One of our managerial suggestions will probably be on this subject and will be addressed directly to SBB.

## **2. Different Knapsack models to optimize**

What to do in the case the travelers aren't interested in all the destinations or have no time/money to cover all the top 10.

- Model with maximum time input from user
- Model with maximum cost input from user

**Assumption**: users are travelling by car for time analysis and by train for costs analysis. This choice has been made to show our adaptation capability in the code which are intially on a similar structure basis. But also because car is a faster mean to travel in Switzerland so it suits the time analysis. Moreover, we encourage tourists to use the train during their travel, it's the best way to preserve the Swiss environment but also to discover the country. Swiss trains are renowned worldwide for their quality (comfort, cleanliness, ...) but also for the landscapes and views offered on board.


### **2.1 Best itinerary with given time from the traveller**
This analysis allows us to define the optimal itinerary according to the tourist's criteria: the importance he attaches to the TripAdvisor ratings of the places visited, the number of days he has available, the daily time he wants to devote to his visits and his starting point among the 10 attractions.





In [24]:
#Importation of the required package for this analysis
import itertools

#Dictionary of concerned destination and their indexes
destination_names = {
    0: "Observation platform of Gornergrat",
    1: "Chillon's castle",
    2: "Jungfraujoch",
    3: "Schweizerischer National Park",
    4: "Lemanic Lake in Lausanne",
    5: "Lugano's town",
    6: "Berne's town",
    7: "Lucerne's town",
    8: "Rhin's waterfalls",
    9: "Zurich's town"}

#Matrix including travel times between our 10 destinations
traveltimes = [[0, 120, 270, 240, 60, 210, 60, 90, 330, 150], #Observation platform of Gornergrat
        [120, 0, 270, 240, 60, 330, 60, 120, 360, 120], #Chillon's castle
        [270, 270, 0, 180, 210, 450, 90, 120, 330, 150], #Jungfraujoch
        [240, 240, 180, 0, 270, 420, 60, 90, 270, 90], #Schweizerischer National Park
        [60, 60, 210, 270, 0, 360, 30, 60, 210, 30], #Lemanic Lake in Lausanne
        [210, 330, 450, 7, 360, 0, 330, 390, 10, 210], #Lugano's town
        [60, 60, 90, 60, 30, 330, 0, 30, 150, 30], #Berne's town
        [90, 120, 120, 90, 60, 390, 30, 0, 210, 30], #Lucerne's town
        [330, 360, 330, 270, 210, 10, 150, 210, 0, 90], #Rhin's waterfalls
        [150, 120, 150, 90, 30, 210, 30, 30, 90, 0]] #Zurich's town

#Visiting times for each of our destinations
visit_times = [2, 2, 4, 4, 1, 2, 2, 2, 2, 2]
#Attraction note for each of our destinations, based on TripAdvisor average note for each of them
attractions = [18, 19, 18, 16, 17, 18, 16, 17, 15, 16]
#Attraction weight in order to define the importance you give to those notations. Set 1 if you don't care about destinations's notes and a 10 if it is your main priority
attraction_weight = 10
#Available traveling days for the tourist 
available_days = 2
#Input here the amount of time you want to spend visiting each day of your travel
daily_visiting_time = 6 * 60 
#Here you can add your starting point between all the 10 destinations
starting_point = 9

#Calculate the total time for a given itinerary
def calculate_total_time(itinerary, traveltimes, visit_times):
    total_time = 0
    for i in range(len(itinerary) - 1):
        travel_time = traveltimes[itinerary[i]][itinerary[i + 1]]
        visit_time = visit_times[itinerary[i]] * 60  #Convert visit time to minutes
        total_time += travel_time + visit_time
    return total_time

#Calculate the total attraction score for a given itinerary
def calculate_total_attraction(itinerary, attractions, attraction_weight):
    return sum(attractions[i] for i in itinerary[:-1]) * attraction_weight 

#Find the best itinerary based on all our constraints
def best_itinerary(traveltimes, attractions, starting_point, available_days, daily_visiting_time, visit_times):
    num_destinations = len(attractions)
    other_destinations = list(range(num_destinations))
    other_destinations.remove(starting_point)
    best_score = float('-inf')
    best_itinerary = None
    max_time = available_days * daily_visiting_time  
    #Generate all subsets of destinations to visit
    for subset_size in range(1, num_destinations):
        for subset in itertools.combinations(other_destinations, subset_size):
            #Generate all possible itineraries for this subset of destinations
            for itinerary in itertools.permutations(subset):
                complete_itinerary = [starting_point] + list(itinerary) + [starting_point]
                total_time = calculate_total_time(complete_itinerary, traveltimes, visit_times)
                if total_time <= max_time:
                    days_needed = total_time // daily_visiting_time
                    if days_needed <= available_days:
                        score = calculate_total_attraction(complete_itinerary, attractions, attraction_weight)
                        if score > best_score:
                            best_score = score
                            best_itinerary = complete_itinerary
    return best_itinerary, best_score

itinerary, score = best_itinerary(
    traveltimes,
    attractions,
    starting_point,
    available_days,
    daily_visiting_time,
    visit_times)

#Convert the itinerary to destination names thanks to their indexes
itinerary_names = [destination_names[i] for i in itinerary]
print("The optimal itinerary according to your demand is:", itinerary_names)
print("With your actual constraints, you are visiting :",len(itinerary_names)-1," places of the top 10 destinations in Switzerland.")
print("This itinerary is based on an attraction weight coefficient of:", attraction_weight)

#Calculate the total time spent on the optimal itinerary
total_time = calculate_total_time(itinerary, traveltimes, visit_times)
print("Total time spent on travelling:", total_time / 60," hours.")  # Convert minutes to hours
final_days = total_time // daily_visiting_time
remaining_minutes = total_time % daily_visiting_time
remaining_hours = remaining_minutes // 60
remaining_minutes = remaining_minutes % 60
print("Total trip duration:", final_days, "days.")
if remaining_hours > 0 or remaining_minutes > 0:
    print(f"Remaining time on the last day: {remaining_hours} hours, {remaining_minutes} minutes.")
else:
    print("No remaining time on the last day.")

The optimal itinerary according to your demand is: ["Zurich's town", "Chillon's castle", 'Lemanic Lake in Lausanne', "Lucerne's town", "Zurich's town"]
With your actual constraints, you are visiting : 4  places of the top 10 destinations in Switzerland.
This itinerary is based on an attraction weight coefficient of: 10
Total time spent on travelling: 11.5  hours.
Total trip duration: 1 days.
Remaining time on the last day: 5 hours, 30 minutes.


**This analysis allows us to inform the user about these different points**:
- The optimal route to travel according to user's constraints
- The number of destinations he can visit among the top 10 destinations in Switzerland with his constraints
- The time spent travelling during the trip
- The total duration of the trip
- The remaining free time at the end of the trip

**Assumptions and hypothesis for this analysis**:
-	The travel times between destinations are symmetric and fixed.
-	The visit times for each destination are given in hours and converted to minutes.
-	Attractions are represented by scores, indicating their level of attractiveness.
-	The attraction weight coefficient determines the importance of attraction scores in the itinerary's calculation.
-	Daily visiting time is specified in minutes.
-	The starting point is one of the destinations, typically Zurich (main Swiss airport).
-	The input data, including travel times, attractions, visit times, attraction weight, available days, daily visiting time, and starting point, are valid and consistent.
-	The objective of the code is to maximize the total attraction score while meeting the given constraints.
-	Visit times for each destination remain constant and are not affected by external factors.
-	Attractions have fixed scores based on TripAdvisor average ratings.
-	The user-defined attraction weight accurately reflects the significance of attraction scores.
-	The daily visiting time remains constant throughout the trip.
-	There are no constraints on the visitation order of destinations, except those defined by the user.
-	Itineraries start and end at the same destination, which is determined by the user as the starting point, typically Zurich (main Swiss airport).

###**2.2 Best itinerary with given budget from the traveller**

This analysis allows us to define the optimal itinerary according to the tourist's criteria: the importance he attaches to the TripAdvisor ratings of the places visited, his budget and his starting point among the 10 attractions.

In [51]:
#Importation of the required package for this analysis
import itertools

#Dictionary of concerned destination and their indexes
destination_names = {
    0: "Observation platform of Gornergrat",
    1: "Chillon's castle",
    2: "Jungfraujoch",
    3: "Schweizerischer National Park",
    4: "Lemanic Lake in Lausanne",
    5: "Lugano's town",
    6: "Berne's town",
    7: "Lucerne's town",
    8: "Rhin's waterfalls",
    9: "Zurich's town"}

#Matrix including travel costs by train between our 10 destinations
costs = [
      [0, 3700, 4150, 8450, 4050, 5350, 4500, 5850, 6850, 6250], # Observation platform of Gornergrat (Zermatt)
      [3700, 0, 3700, 6200, 650, 5600, 2100, 3600, 4400, 4000], # Chillon's castle (Montreux)
      [4150, 3700, 0, 5800, 2800, 4200, 1400, 1650, 4000, 2550], # Jungfraujoch (Interlaken Ost)
      [8450, 6200, 5800, 0, 6000, 5600, 5000, 3700, 3600, 3150], # Schweizerischer National Park (Zernez puis Bus à 2CHF)
      [4050, 650, 2800, 6000, 0, 5400, 1700, 3250, 4200, 3700], # Lemanic Lake in Lausanne
      [5350, 5600, 4200, 5600, 5400, 0, 4400, 3050, 3800, 3250], # Lugano's town
      [4500, 2100, 1400, 5000, 1700, 4400, 0, 1950, 3050, 2550], # Berne's town
      [5850, 3600, 1650, 3700, 3250, 3050, 1950, 0, 3700, 2500], # Lucerne's town
      [6850, 4400, 4000, 3600, 4200, 3800, 3050, 3700, 0, 2320], # Rhin's waterfalls (Neuhasen Rheinfall)
      [6250, 4000, 2550, 3150, 3700, 3250, 2550, 2500, 2320, 0]] #Zurich's town

#Visiting costs for each of our destinations
visit_costs = [55, 13, 50, 9, 50, 40, 40, 40, 55, 50]
#Attraction note for each of our destinations, based on TripAdvisor average note for each of them
attractions = [18, 19, 18, 16, 17, 18, 16, 17, 15, 16]
#Attraction weight in order to define the importance you give to those notations. Set 1 if you don't care about destinations's notes and a 10 if it is your main priority
attraction_weight = 10
#Input your budget in CHF
available_budget = 400
#Here you can add your starting point between all the 10 destinations
starting_point = 9
#Take the half-fare train card
fare = True

if fare == True:
  available_budget = (available_budget-185)*100
else : 
  available_budget = available_budget*50  

#Calculate the total cost for a given itinerary
def calculate_total_cost(itinerary, costs, visit_costs):
    total_cost = sum(visit_costs[i] for i in itinerary[:-1])
    for i in range(len(itinerary) - 1):
        total_cost += costs[itinerary[i]][itinerary[i+1]]
    return total_cost

#Calculate the total attraction score for a given itinerary
def calculate_total_attraction(itinerary, attractions, attraction_weight):
    return sum(attractions[i] for i in itinerary[:-1]) * attraction_weight 

#Find the best itinerary based on all our constraints
def best_itinerary(costs, visit_costs, starting_point, available_budget, attractions, attraction_weight):
    num_destinations = len(costs)
    other_destinations = list(range(num_destinations))
    other_destinations.remove(starting_point)
    best_score = float('-inf')
    best_itinerary = None
    #Generate all subsets of destinations to visit
    for subset_size in range(1, num_destinations):
        for subset in itertools.combinations(other_destinations, subset_size):
            #Generate all possible itineraries for this subset of destinations
            for itinerary in itertools.permutations(subset):
                complete_itinerary = [starting_point] + list(itinerary) + [starting_point]
                total_cost = sum(costs[complete_itinerary[i]][complete_itinerary[i+1]] for i in range(len(complete_itinerary)-1))
                total_visit_cost = sum(visit_costs[i] for i in complete_itinerary[:-1])
                if total_cost + total_visit_cost <= available_budget:
                    score = calculate_total_attraction(complete_itinerary, attractions, attraction_weight)
                    if score > best_score:
                        best_score = score
                        best_itinerary = complete_itinerary
    return best_itinerary, best_score

itinerary, best_score = best_itinerary(
    costs,
    visit_costs,
    starting_point,
    available_budget,
    attractions,
    attraction_weight)


#Convert the itinerary to destination names thanks to their indexes
itinerary_names = [destination_names[i] for i in itinerary]
print("The optimal itinerary according to your budget is:\n",itinerary_names,"\n and is based on an attraction weight coeff. of:",attraction_weight,"on a scale of 10.")
#Calculate the total time spent on the optimal itinerary
total_cost = calculate_total_cost(itinerary, costs, visit_costs)
tot_cost = total_cost/100
av_budg = available_budget/100
if fare == True:
  print("With your",av_budg+185,"CHF personal budget, you can thus visit",len(itinerary_names)-1,"different destinations.")
  print("Total cost of the trip is :", tot_cost,"CHF + the price of the half-fare card, it finally gives :", tot_cost+185,"CHF.")
  print("Remaining personal budget after the trip:",(available_budget-total_cost)/100,"CHF.")
  if av_budg < 185 :
    print("Since the cost of your trip is:",tot_cost,"CHF which is < 185 CHF, we suggest you not to buy the half-fare card, you will then probably be able to make a bigger travel.")
else :
  print("With your",(available_budget/100)*2,"CHF personal budget, you can thus visit",len(itinerary_names)-1,"different destinations.")
  print("Total cost of the trip is :", tot_cost*2,"CHF.")
  print("Remaining personal budget after the trip:",(available_budget-total_cost)/50,"CHF.")
  if av_budg > 184 :
    print("Since half the cost of your trip is:",tot_cost,"CHF which is >= 185 CHF, we suggest you to buy the half-fare card, you will then probably be able to make a bigger travel.")


The optimal itinerary according to your budget is:
 ["Zurich's town", 'Jungfraujoch', 'Observation platform of Gornergrat', "Chillon's castle", 'Lemanic Lake in Lausanne', "Berne's town", "Lucerne's town", "Lugano's town", "Zurich's town"] 
 and is based on an attraction weight coeff. of: 10 on a scale of 10.
With your 400.0 CHF personal budget, you can thus visit 8 different destinations.
Total cost of the trip is : 213.38 CHF + the price of the half-fare card, it finally gives : 398.38 CHF.
Remaining personal budget after the trip: 1.62 CHF.


**This analysis allows us to inform the user about these different points**:
- The optimal route to travel according to user's constraints
- The number of destinations he can visit among the top 10 destinations in Switzerland with his budget
- The total cost of the trip
- The remaining budget at the end of the trip
- If he is making the right choice by taking the half-fare card or not

**Assumptions**:
- Costs are given *100 in the costs matrix, otherwise the code isn't running
-	The travel costs between destinations are symmetric and fixed.
-	The visit costs for each destination are constant and independent of factors such as season or time.
-	Attractions have fixed scores based on TripAdvisor average ratings.
-	The user-defined attraction weight coefficient accurately represents the importance of attraction scores in itinerary planning.
-	The available budget is given in CHF (Swiss Francs).
-	All destinations can be visited in any order, and there are no dependencies or constraints on the visitation order, except those defined by the user.
-	Itineraries start and end at the same destination, typically Zurich (main Swiss airport).
-	The code aims to find the best itinerary that maximizes the total attraction score while respecting the budget constraint.
-	The remaining budget after the trip is calculated as the available budget minus the total cost of the trip.
-	The travel costs and visit costs are fixed and known beforehand, without any variations based on external factors.
-	The attraction score is based on TripAdvisor's ratings for each location.
-	The itinerary always starts and ends at the same location, which is determined by the user as the starting point.
-	The itinerary can include any combination of destinations without constraints on the order or subset of destinations.
-	The best itinerary is determined by the highest attraction score within the given budget constraint.